In [5]:
import pandas as pd
import matplotlib.pyplot as plt

def analizar_datos_demograficos(ruta_archivo='datos_personales.csv'):
    """
    Función para analizar datos demográficos desde un archivo CSV
    """
    # Leer el archivo CSV con punto y coma como separador
    df = pd.read_csv(ruta_archivo, sep=';')
    
    # Análisis estadístico expandido
    resultados = {
        # Análisis por edad
        'estadisticas_edad': {
            'promedio': df['Edad'].mean(),
            'mediana': df['Edad'].median(),
            'max': df['Edad'].max(),
            'min': df['Edad'].min(),
            'desviacion_estandar': df['Edad'].std(),
            'cuartiles': df['Edad'].quantile([0.25, 0.5, 0.75]).to_dict()
        },
        
        # Distribución por sexo
        'distribucion_sexo': df['Sexo'].value_counts().to_dict(),
        'porcentaje_sexo': (df['Sexo'].value_counts(normalize=True) * 100).to_dict(),
        
        # Distribución por barrio
        'personas_por_barrio': df['Barrio'].value_counts().to_dict(),
        'porcentaje_barrio': (df['Barrio'].value_counts(normalize=True) * 100).to_dict(),
        
        # Rangos de edad más detallados
        'rangos_edad': pd.cut(df['Edad'], 
                            bins=[0, 12, 18, 25, 35, 45, 55, 65, 75, 100],
                            labels=['0-12', '13-18', '19-25', '26-35', '36-45', 
                                   '46-55', '56-65', '66-75', '75+']).value_counts().to_dict(),
        
        # Análisis de contacto
        'estadisticas_contacto': {
            'con_email': (df['Email'].notna().sum() / len(df) * 100),
            'con_celular': (df['Numero Celular'].notna().sum() / len(df) * 100)
        },
        
        # Análisis cruzado barrio-sexo
        'distribucion_barrio_sexo': pd.crosstab(df['Barrio'], df['Sexo']).to_dict()
    }
    
    # Crear visualizaciones
    crear_visualizaciones(df)
    
    return resultados

def crear_visualizaciones(df):
    """
    Crear visualizaciones usando solo matplotlib
    """
    # 1. Histograma de edades por sexo
    plt.figure(figsize=(12, 6))
    for sexo in df['Sexo'].unique():
        plt.hist(df[df['Sexo'] == sexo]['Edad'], 
                alpha=0.5, 
                label=sexo, 
                bins=20)
    plt.title('Distribución de Edades por Sexo')
    plt.xlabel('Edad')
    plt.ylabel('Cantidad de Personas')
    plt.legend()
    plt.savefig('distribucion_edades_sexo.png')
    plt.close()
    
    # 2. Gráfico de barras para distribución por barrio
    plt.figure(figsize=(14, 6))
    barrios = df['Barrio'].value_counts()
    plt.bar(range(len(barrios)), barrios.values)
    plt.xticks(range(len(barrios)), barrios.index, rotation=45)
    plt.title('Distribución por Barrio')
    plt.xlabel('Barrio')
    plt.ylabel('Cantidad de Personas')
    plt.tight_layout()
    plt.savefig('distribucion_barrios.png')
    plt.close()
    
    # 3. Gráfico de barras apiladas para distribución barrio-sexo
    plt.figure(figsize=(14, 6))
    pd.crosstab(df['Barrio'], df['Sexo']).plot(kind='bar', stacked=True)
    plt.title('Distribución por Barrio y Sexo')
    plt.xlabel('Barrio')
    plt.ylabel('Cantidad de Personas')
    plt.legend(title='Sexo')
    plt.tight_layout()
    plt.savefig('distribucion_barrio_sexo.png')
    plt.close()
    
    # 4. Box plot de edades por barrio
    plt.figure(figsize=(14, 6))
    df.boxplot(column='Edad', by='Barrio', figsize=(14, 6))
    plt.title('Distribución de Edades por Barrio')
    plt.suptitle('')  # Esto elimina el título automático del boxplot
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('edades_por_barrio.png')
    plt.close()

def generar_reporte(resultados):
    """
    Genera un reporte con los resultados del análisis
    """
    reporte = f"""
REPORTE DE ANÁLISIS DEMOGRÁFICO DETALLADO
{'-' * 50}

1. ESTADÍSTICAS DE EDAD:
   - Edad promedio: {resultados['estadisticas_edad']['promedio']:.1f} años
   - Edad mediana: {resultados['estadisticas_edad']['mediana']:.1f} años
   - Edad máxima: {resultados['estadisticas_edad']['max']} años
   - Edad mínima: {resultados['estadisticas_edad']['min']} años
   - Desviación estándar: {resultados['estadisticas_edad']['desviacion_estandar']:.1f} años
   - Cuartiles:
     * Q1 (25%): {resultados['estadisticas_edad']['cuartiles'][0.25]:.1f} años
     * Q2 (50%): {resultados['estadisticas_edad']['cuartiles'][0.5]:.1f} años
     * Q3 (75%): {resultados['estadisticas_edad']['cuartiles'][0.75]:.1f} años

2. DISTRIBUCIÓN POR SEXO:
   {'-' * 40}"""
    
    for sexo, cantidad in resultados['distribucion_sexo'].items():
        porcentaje = resultados['porcentaje_sexo'][sexo]
        reporte += f"\n   - {sexo}: {cantidad} personas ({porcentaje:.1f}%)"
    
    reporte += f"\n\n3. DISTRIBUCIÓN POR RANGOS DE EDAD:\n   {'-' * 40}"
    for rango, cantidad in resultados['rangos_edad'].items():
        reporte += f"\n   - {rango}: {cantidad} personas"
    
    reporte += f"\n\n4. DISTRIBUCIÓN POR BARRIO:\n   {'-' * 40}"
    for barrio, cantidad in resultados['personas_por_barrio'].items():
        porcentaje = resultados['porcentaje_barrio'][barrio]
        reporte += f"\n   - {barrio}: {cantidad} personas ({porcentaje:.1f}%)"
    
    reporte += f"\n\n5. ESTADÍSTICAS DE CONTACTO:\n   {'-' * 40}"
    reporte += f"""
   - Porcentaje con email: {resultados['estadisticas_contacto']['con_email']:.1f}%
   - Porcentaje con celular: {resultados['estadisticas_contacto']['con_celular']:.1f}%

6. VISUALIZACIONES GENERADAS:
   {'-' * 40}
   Se han generado los siguientes archivos:
   - distribucion_edades_sexo.png: Histograma de edades por sexo
   - distribucion_barrios.png: Distribución por barrio
   - distribucion_barrio_sexo.png: Distribución por barrio y sexo
   - edades_por_barrio.png: Distribución de edades por barrio
"""
    return reporte

if __name__ == "__main__":
    resultados = analizar_datos_demograficos()
    reporte = generar_reporte(resultados)
    print(reporte)


REPORTE DE ANÁLISIS DEMOGRÁFICO DETALLADO
--------------------------------------------------

1. ESTADÍSTICAS DE EDAD:
   - Edad promedio: 37.8 años
   - Edad mediana: 36.0 años
   - Edad máxima: 74 años
   - Edad mínima: 6 años
   - Desviación estándar: 17.5 años
   - Cuartiles:
     * Q1 (25%): 25.8 años
     * Q2 (50%): 36.0 años
     * Q3 (75%): 52.2 años

2. DISTRIBUCIÓN POR SEXO:
   ----------------------------------------
   - M: 92 personas (52.3%)
   - F: 84 personas (47.7%)

3. DISTRIBUCIÓN POR RANGOS DE EDAD:
   ----------------------------------------
   - 26-35: 41 personas
   - 36-45: 28 personas
   - 56-65: 28 personas
   - 46-55: 27 personas
   - 0-12: 20 personas
   - 19-25: 14 personas
   - 13-18: 10 personas
   - 66-75: 8 personas
   - 75+: 0 personas

4. DISTRIBUCIÓN POR BARRIO:
   ----------------------------------------
   - SIERRA ALTA: 154 personas (87.5%)
   - Sierra Alta: 22 personas (12.5%)

5. ESTADÍSTICAS DE CONTACTO:
   -----------------------------------

<Figure size 1400x600 with 0 Axes>

<Figure size 1400x600 with 0 Axes>